In [1]:
import os,sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tqdm

In [2]:
#File Load
pipline ="\\"
#if linux piplin="/"
basepath = "DataSet(190408)/"
path = [os.path.join(basepath, x) for x in os.listdir(basepath)] 
path2 = [os.path.join(x, y) for x in path if os.path.isdir(x) for y in os.listdir(x)]
path3= [os.path.join(x, y) for x in path2 if x[-6:] =="_Truth" for y in os.listdir(x)]
path2 = [os.path.join(x, y) for x in path3 if os.path.isdir(x) for y in os.listdir(x)]
path3 = [os.path.join(x, y) for x in path2 if os.path.isdir(x) for y in os.listdir(x) if y[-3:] =="txt"]
FilePath = [[x.replace("Ground_Truth", "Parse_Data").replace("txt","jpg"), x ]for x in path3 ]

In [3]:
def find_ball(label):
    ball_position = []
    f = open(label,"r")
    while True:
        line = f.readline()
        if not line: break
        if line.split(",")[0].split("\'")[1] =='sports ball':
            try:
                ball_position.append([int(x) for x in line.split(",")[1:5]])
            except ValueError:
                ball_position.append([int(x) for x in line.split(",")[1:4]] + [int(line.split(",")[4][:-2])])
    return ball_position

In [ ]:
def make_gtmap(ball_position_list):
    label_map1 = np.zeros((1080,1920,1), dtype = np.uint8)
    label_map2 = np.zeros((1080,1920,1), dtype = np.uint8)
    label_map3 = np.zeros((1080,1920,1), dtype = np.uint8)
    label_map4 = np.zeros((1080,1920,1), dtype = np.uint8)
    for ball_position in ball_position_list:
        label_map1[ball_position[1]:ball_position[1] + ball_position[3], ball_position[0]:ball_position[0] +ball_position[2],:] = 255
        label_map2[-ball_position[1]:-ball_position[1] - ball_position[3], -ball_position[0]:-ball_position[0] -ball_position[2],:] = 255
        label_map3[-ball_position[1]:-ball_position[1] - ball_position[3], ball_position[0]:ball_position[0] +ball_position[2],:] = 255
        label_map4[ball_position[1]:ball_position[1] + ball_position[3], -ball_position[0]:-ball_position[0] -ball_position[2],:] = 255
        
    return [label_map1,label_map2,label_map3,label_map4]
    

In [ ]:
def image_load_and_batch(img_name):
    
    image_map = []
    img =cv2.imread(img_name)

    #ori
    img_map = np.zeros((1080,1920,3), dtype=np.uint8)
    img_map[0:img.shape[0],0:img.shape[1],0:img.shape[2]] = img
    
    #rb
    
    img_map2 = np.zeros((1080,1920,3), dtype=np.uint8)
    img_map2[-img.shape[0]:,-img.shape[1]:,-img.shape[2]:] = img

    
    #lb
    img_map3 = np.zeros((1080,1920,3), dtype=np.uint8)
    img_map3[-img.shape[0]:,0:img.shape[1],:] = img

    
    #rt
    img_map4= np.zeros((1080,1920,3), dtype=np.uint8)
    img_map4[0:img.shape[0],-img.shape[1]:,:] = img


    
    return [img_map,img_map2,img_map3,img_map4]

In [ ]:
def data_argumentation(data, data2):
    
    noise = tf.random_normal(shape=tf.shape(data), mean=0.0, stddev=1.0,dtype=tf.float32)
    noise = tf.random_normal(shape=tf.shape(data), mean=0.0, stddev=1.0,dtype=tf.float32)
    noise2 = tf.random_gamma(shape=tf.shape(data)[0:2], alpha = [0,1,2], dtype=tf.float32)
    
    flip_ud = tf.image.flip_up_down(data)
    flip_lr = tf.image.flip_left_right(data)
    rot_90 = tf.image.resize_images(tf.image.rot90(data, k=1), (1080,1920))
    rot_180 = tf.image.rot90(data, k=2)
    rot_270 = tf.image.resize_images(tf.image.rot90(data, k=3), (1080,1920))
    gau = tf.add(data,noise)
    gamma = tf.image.adjust_gamma(data)
    
    flip_ud2 = tf.image.flip_up_down(data2)
    flip_lr2 = tf.image.flip_left_right(data2)
    rot_902 = tf.image.resize_images(tf.image.rot90(data2, k=1), (1080,1920))
    rot_1802 = tf.image.rot90(data2, k=2)
    rot_2702 = tf.image.resize_images(tf.image.rot90(data2, k=1), (1080,1920))
    gau2 = tf.add(data2,noise)
    gamma2 = tf.image.adjust_gamma(data2)
    
    return flip_ud, flip_lr, rot_90, rot_180, rot_270, gau, gamma, flip_ud2, flip_lr2, rot_902,rot_1802, rot_2702, gau2, gamma2
    
    

In [ ]:
init_op = tf.initialize_all_variables()
tf.reset_default_graph()
X = tf.placeholder(tf.float32,[None, None, None])
Y = tf.placeholder(tf.float32,[None,None,None])

iflip_ud, iflip_lr, irot_90, irot_180, irot_270, igau, igmma, lflip_ud, lflip_lr, lrot_90, lrot_180, lrot_270, lgau, lgmma=data_argumentation(X, Y)


with tf.Session(config=tf.ConfigProto(gpu_options=(tf.GPUOptions(per_process_gpu_memory_fraction=1.0)))) as sess:
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for i,File in enumerate(FilePath[:2000]):
        
        
            img, label = File[0], File[1]
            print(label)
            ball_posi = find_ball(label)
            ball_maps = make_gtmap(ball_posi)
            img_maps = image_load_and_batch(img)
            
            for n, img_map in enumerate(img_maps):
                
                try:
                    ball_map = ball_maps[n]

                    folder_path1 = os.path.join(File[1].split(pipline)[0],"Deep_pre")
                    folder_path2 = os.path.join(File[1].split(pipline)[0],"Deep_pre", File[1].split(pipline)[2])
                    folder_path3 = os.path.join(File[1].split(pipline)[0],"Deep_pre", File[1].split(pipline)[2], File[1].split(pipline)[3])
                    #folder_path3 = os.path.join(File[1].split(pipline)[0],"Deep_pre", File[1].split[1](pipline)[2], File.split[1](pipline)[3])

                    if not os.path.exists(folder_path1): os.mkdir(folder_path1)
                    if not os.path.exists(folder_path2): os.mkdir(folder_path2)
                    if not os.path.exists(folder_path3): os.mkdir(folder_path3)
                    if not os.path.exists(os.path.join(folder_path3, "img")): os.mkdir(os.path.join(folder_path3, "img"))
                    if not os.path.exists(os.path.join(folder_path3, "label")): os.mkdir(os.path.join(folder_path3, "label"))

                    niflip_ud, niflip_lr, nirot_90, nirot_180, nirot_270, nigau, nigamma, nlflip_ud, nlflip_lr, nlrot_90, nlrot_180, nlrot_270, nlgau, nlgamma = sess.run([iflip_ud, iflip_lr, irot_90, irot_180, irot_270, igau, igmma, lflip_ud, lflip_lr, lrot_90, lrot_180, lrot_270, lgau, lgmma], feed_dict={X:img_map, Y:ball_map})

                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "flipud.jpg"), np.array(niflip_ud, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "rot_90.jpg"), np.array(nirot_90, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "rot_180.jpg"), np.array(nirot_180, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "rot_270.jpg"), np.array(nirot_270, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "gau.jpg"), np.array(nigau, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "gamma.jpg"), np.array(nigamma, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"img",str(i)+str(n)+ "ori.jpg"), np.array(img_map, dtype="uint8"))

                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "flipud.jpg"), np.array(nlflip_ud, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "fliplr.jpg"), np.array(nlflip_lr, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "rot_90.jpg"), np.array(nlrot_90, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "rot_180.jpg"), np.array(nlrot_180, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "rot_270.jpg"), np.array(nlrot_270, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "gau.jpg"), np.array(ball_map, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "gamma.jpg"), np.array(nlgamma, dtype="uint8"))
                    cv2.imwrite(os.path.join(folder_path3,"label",str(i)+str(n)+ "ori.jpg"), np.array(ball_map, dtype="uint8"))


                except ValueError:
                    pass





    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
DataSet(190408)/Free_A\Ground_Truth\00000\0\F1.txt
DataSet(190408)/Free_A\Ground_Truth\00000\0\F2.txt
DataSet(190408)/Free_A\Ground_Truth\00000\0\F3.txt
DataSet(190408)/Free_A\Ground_Truth\00000\1\1_10.txt
DataSet(190408)/Free_A\Ground_Truth\00000\1\1_11.txt
DataSet(190408)/Free_A\Ground_Truth\00000\1\1_12.txt
DataSet(190408)/Free_A\Ground_Truth\00000\2\1_13.txt
DataSet(190408)/Free_A\Ground_Truth\00000\2\1_18.txt
DataSet(190408)/Free_A\Ground_Truth\00000\2\1_24.txt
DataSet(190408)/Free_A\Ground_Truth\00000\3\1_25.txt
DataSet(190408)/Free_A\Ground_Truth\00000\3\1_29.txt
DataSet(190408)/Free_A\Ground_Truth\00000\3\1_33.txt
DataSet(190408)/Free_A\Ground_Truth\00001\0\F4.txt
DataSet(190408)/Free_A\Ground_Truth\00001\0\F5.txt
DataSet(190408)/Free_A\Ground_Truth\00001\0\F6.txt
DataSet(190408)/Free_A\Ground_Truth\00001\1\2_10.txt
DataSet(190408)/Free_A\Ground_Truth\00001\1\2_8.txt
DataSet(190408)/Free_A\Ground_Truth\00

DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (157).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (158).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (159).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (160).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (161).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (162).txt
DataSet(190408)/Free_A\Ground_Truth\00009\0\Free1 (163).txt
DataSet(190408)/Free_A\Ground_Truth\00009\1\Free1 (164).txt
DataSet(190408)/Free_A\Ground_Truth\00009\1\Free1 (165).txt
DataSet(190408)/Free_A\Ground_Truth\00009\1\Free1 (166).txt
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (167).txt
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (168).txt
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (169).txt
DataSet(190408)/Free_A\Ground_Truth\00009\2\Free1 (170).txt
DataSet(190408)/Free_A\Ground_Truth\00009\3\Free1 (171).txt
DataSet(190408)/Free_A\Ground_Truth\00009\3\Free1 (172).txt
DataSet(190408)/Free_A\Ground_Truth\0000

DataSet(190408)/Free_A\Ground_Truth\00018\1\00038.txt
DataSet(190408)/Free_A\Ground_Truth\00018\2\00040.txt
DataSet(190408)/Free_A\Ground_Truth\00018\2\00060.txt
DataSet(190408)/Free_A\Ground_Truth\00018\2\00080.txt
DataSet(190408)/Free_A\Ground_Truth\00018\3\00081.txt
DataSet(190408)/Free_A\Ground_Truth\00018\3\00086.txt
DataSet(190408)/Free_A\Ground_Truth\00018\3\00091.txt
DataSet(190408)/Free_A\Ground_Truth\00019\0\00000.txt
DataSet(190408)/Free_A\Ground_Truth\00019\0\00009.txt
DataSet(190408)/Free_A\Ground_Truth\00019\0\00018.txt
DataSet(190408)/Free_A\Ground_Truth\00019\1\00019.txt
DataSet(190408)/Free_A\Ground_Truth\00019\1\00021.txt
DataSet(190408)/Free_A\Ground_Truth\00019\1\00023.txt
DataSet(190408)/Free_A\Ground_Truth\00019\2\00025.txt
DataSet(190408)/Free_A\Ground_Truth\00019\2\00031.txt
DataSet(190408)/Free_A\Ground_Truth\00019\2\00038.txt
DataSet(190408)/Free_A\Ground_Truth\00019\3\00040.txt
DataSet(190408)/Free_A\Ground_Truth\00019\3\00043.txt
DataSet(190408)/Free_A\Groun

DataSet(190408)/Free_A\Ground_Truth\00028\3\00757.txt
DataSet(190408)/Free_A\Ground_Truth\00029\0\00924.txt
DataSet(190408)/Free_A\Ground_Truth\00029\0\00931.txt
DataSet(190408)/Free_A\Ground_Truth\00029\0\00938.txt
DataSet(190408)/Free_A\Ground_Truth\00029\1\00939.txt
DataSet(190408)/Free_A\Ground_Truth\00029\1\00942.txt
DataSet(190408)/Free_A\Ground_Truth\00029\1\00946.txt
DataSet(190408)/Free_A\Ground_Truth\00029\2\00947.txt
DataSet(190408)/Free_A\Ground_Truth\00029\2\00960.txt
DataSet(190408)/Free_A\Ground_Truth\00029\2\00968.txt
DataSet(190408)/Free_A\Ground_Truth\00029\3\00969.txt
DataSet(190408)/Free_A\Ground_Truth\00029\3\00972.txt
DataSet(190408)/Free_A\Ground_Truth\00029\3\00975.txt
DataSet(190408)/Free_A\Ground_Truth\00030\0\01098.txt
DataSet(190408)/Free_A\Ground_Truth\00030\0\01106.txt
DataSet(190408)/Free_A\Ground_Truth\00030\0\01113.txt
DataSet(190408)/Free_A\Ground_Truth\00030\1\01114.txt
DataSet(190408)/Free_A\Ground_Truth\00030\1\01117.txt
DataSet(190408)/Free_A\Groun

DataSet(190408)/Free_A\Ground_Truth\00041\2\72.txt
DataSet(190408)/Free_A\Ground_Truth\00041\3\101.txt
DataSet(190408)/Free_A\Ground_Truth\00041\3\106.txt
DataSet(190408)/Free_A\Ground_Truth\00041\3\113.txt
DataSet(190408)/Free_A\Ground_Truth\00042\0\1905.txt
DataSet(190408)/Free_A\Ground_Truth\00042\0\1919.txt
DataSet(190408)/Free_A\Ground_Truth\00042\0\1933.txt
DataSet(190408)/Free_A\Ground_Truth\00042\1\1934.txt
DataSet(190408)/Free_A\Ground_Truth\00042\1\1939.txt
DataSet(190408)/Free_A\Ground_Truth\00042\1\1944.txt
DataSet(190408)/Free_A\Ground_Truth\00042\2\1945.txt
DataSet(190408)/Free_A\Ground_Truth\00042\2\1964.txt
DataSet(190408)/Free_A\Ground_Truth\00042\2\1983.txt
DataSet(190408)/Free_A\Ground_Truth\00042\3\1984.txt
DataSet(190408)/Free_A\Ground_Truth\00042\3\1994.txt
DataSet(190408)/Free_A\Ground_Truth\00042\3\2004.txt
DataSet(190408)/Free_A\Ground_Truth\00043\0\2490.txt
DataSet(190408)/Free_A\Ground_Truth\00043\0\2493.txt
DataSet(190408)/Free_A\Ground_Truth\00043\0\2496.tx

DataSet(190408)/Free_B\Ground_Truth\00054\1\05152.txt
DataSet(190408)/Free_B\Ground_Truth\00054\2\05153.txt
DataSet(190408)/Free_B\Ground_Truth\00054\2\05168.txt
DataSet(190408)/Free_B\Ground_Truth\00054\2\05184.txt
DataSet(190408)/Free_B\Ground_Truth\00054\3\05185.txt
DataSet(190408)/Free_B\Ground_Truth\00054\3\05195.txt
DataSet(190408)/Free_B\Ground_Truth\00054\3\05205.txt
DataSet(190408)/Free_B\Ground_Truth\00055\0\05343.txt
DataSet(190408)/Free_B\Ground_Truth\00055\0\05360.txt
DataSet(190408)/Free_B\Ground_Truth\00055\0\05377.txt
DataSet(190408)/Free_B\Ground_Truth\00055\1\05378.txt
DataSet(190408)/Free_B\Ground_Truth\00055\1\05382.txt
DataSet(190408)/Free_B\Ground_Truth\00055\1\05386.txt
DataSet(190408)/Free_B\Ground_Truth\00055\2\05387.txt
DataSet(190408)/Free_B\Ground_Truth\00055\2\05411.txt
DataSet(190408)/Free_B\Ground_Truth\00055\2\05436.txt
DataSet(190408)/Free_B\Ground_Truth\00055\3\05437.txt
DataSet(190408)/Free_B\Ground_Truth\00055\3\05444.txt
DataSet(190408)/Free_B\Groun

In [ ]:
i.mage_load_and_batch("DataSet(190408)/Free_A\Parse_Data\00012\3\Free1 (285).txt")